<a href="https://colab.research.google.com/github/Ruijia0/SupplyChain_Analytics/blob/main/Tailored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math


A_0 = 150  # Common (fixed) ordering cost for placing any order
products = [
    # Each entry is: (Annual Demand, Product-specific ordering cost, Unit cost, Holding rate)
    {"D": 1000, "A_i": 20, "c": 50, "h": 0.15},  # Product 1
    {"D":  300, "A_i": 25, "c": 60, "h": 0.15},  # Product 2
    {"D":  100, "A_i": 30, "c": 30, "h": 0.15},  # Product 3
    {"D":   50, "A_i": 50, "c": 30, "h": 0.15},  # Product 4
]

# A handy function: S_i = D_i * c_i * h_i for each product
def holding_multiplier(p):
    return p["D"] * p["c"] * p["h"]

# 1) Products Sourced Independently
# For each product i, cost is:
#     Cost_i = sqrt( 2 * (A_0 + A_i) * (D_i * c_i * h_i) )
# Then sum across all products.

cost_independent = 0.0
for p in products:
    A_total = A_0 + p["A_i"]         # Common + specific cost
    S_i = holding_multiplier(p)      # = D_i*c_i*h_i
    cost_independent += math.sqrt(2 * A_total * S_i)

print("1) All products sourced independently:")
print(f"   Total annual cost ≈ {cost_independent:.2f}\n")


# 2) All Four Products Sourced with the Same Frequency
# If all 4 are in one group, each order costs (A_0 + sum of A_i),
# and total S = sum of D_i*c_i*h_i.
# The optimal number of orders per year is n* = sqrt( sum(S_i) / [2*(A_0 + sum(A_i))] ).
# Total cost = (A_0 + sum(A_i)) * n* + sum(S_i)/(2*n*).

sum_Ai = sum(p["A_i"] for p in products)
sum_Si = sum(holding_multiplier(p) for p in products)

n_star = math.sqrt(sum_Si / (2.0 * (A_0 + sum_Ai)))
cost_all_together = (A_0 + sum_Ai) * n_star + (sum_Si / (2.0 * n_star))

print("2) All four products sourced together (same frequency):")
print(f"   Optimal annual cost ≈ {cost_all_together:.2f}\n")


# 3) Tailored (Selective) Aggregation
# In principle, one can form any partition of the set {1,2,3,4} into groups,
# and order each group at its own frequency. The cost for each group G is:
#     group_cost(G) = sqrt( 2*(A_0 + sum(A_i for i in G)) * sum(S_i for i in G) )
# Then total cost is the sum of group_cost(G) over all groups in the partition.
#
# For just 4 products, we can brute force all partitions. However, from theory,
# the minimum turns out to be the same as "all together." But below is a
# general partition-based search if you want to confirm or extend it.

# We'll label products by indices [0,1,2,3] in code.

def group_cost(group, products):
    """Compute the cost for a single group of product indices."""
    sum_Ai_group = sum(products[i]["A_i"] for i in group)
    sum_Si_group = sum(holding_multiplier(products[i]) for i in group)
    return math.sqrt(2.0 * (A_0 + sum_Ai_group) * sum_Si_group)

def all_partitions(collection):
    """
    Generate all partitions of the given list 'collection' into non-empty subsets.
    For example, for [0,1,2], it yields:
       [[0,1,2]],
       [[0],[1,2]], [[1],[0,2]], [[2],[0,1]],
       [[0],[1],[2]].
    """
    if len(collection) == 1:
        yield [collection]
        return

    first = collection[0]
    for partial in all_partitions(collection[1:]):
        # 1) Put 'first' in its own subset
        yield [[first]] + partial
        # 2) Or add 'first' to any existing subset
        for i in range(len(partial)):
            new_partition = partial[:i] + [[first] + partial[i]] + partial[i+1:]
            yield new_partition

indices = [0,1,2,3]
min_cost = float("inf")
best_partition = None

for partition in all_partitions(indices):
    total = 0.0
    for group in partition:
        total += group_cost(group, products)
    if total < min_cost:
        min_cost = total
        best_partition = partition

print("3) Tailored aggregation (optimal partition search):")
print(f"   Minimum annual cost ≈ {min_cost:.2f}")
print("   Best partition of products (0-based indices):", best_partition)

# Note that 'best_partition' should come out as all products in a single group
# (i.e., [[0,1,2,3]]) and the cost ~ 2,446, matching scenario (2).


1) All products sourced independently:
   Total annual cost ≈ 3271.48

2) All four products sourced together (same frequency):
   Optimal annual cost ≈ 2445.66

3) Tailored aggregation (optimal partition search):
   Minimum annual cost ≈ 2445.66
   Best partition of products (0-based indices): [[0, 1, 2, 3]]
